In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://10.0.2.180:27017')
xiaomu = client.xiaomu
message = xiaomu.message

In [4]:
course_id = 'course-v1:TsinghuaX+30240184+sp'

In [29]:
message_set = message.find(
        {'course_id': course_id, 'flag': {"$in": [None, 'more']}, 'question_source': {"$nin": ['wobudong', 'active_question']}}).sort('_id', -1)

In [30]:
q_dict, a_list = {}, []
for m in message_set:
    if 'message' not in m.keys():        
        continue

    if m['type'] == 'question':
        q_dict[m['_id']] = m

    if m['type'] == 'answer':
        a_list.append(m)

print('q_dict len: %d, a_list len: %d' % (len(q_dict), len(a_list)))

q_dict len: 16628, a_list len: 5658


In [31]:
labeled_questions = set(
    [x['question'] for x in xiaomu.qa_annotation.find()])
len(labeled_questions)

1514

In [32]:
qid_list, q_text, a_text, times, tags = [], [], [], [], []
for v in a_list:
    if 'origin_question' not in v:
        continue

    q_id = v['origin_question']
    if q_id not in q_dict:
        continue

    if q_dict[q_id]['message'] in labeled_questions:
        continue

    if '[    ]' in q_dict[q_id]['message']:
        continue

    qid_list.append(q_id)
    q_text.append(q_dict[q_id]['message'])
    a_text.append(v['message'] if v['message'] else (
        v['answers'][0]['message'] if 'answers' in v else ''))

    times.append(v['time'])
    tags.append(v.get('tag', -1))

response = [qid_list, a_text, q_text, times, tags]
len(set(q_text))

877

In [20]:
message_set = xiaomu.message.find(
            {'course_id': course_id, 'type': 'question', 'flag': {"$in": [None, 'more']}, 'question_source': {"$nin": ['wobudong', 'active_question']}})
message_set = list(message_set)
len(message_set)

16628

In [25]:

items = xiaomu.message.find({'course_id': course_id, 'type': 'answer', 'flag': {"$in": [None, 'more']}})
items = list(items)
print(len(items))
items = list(filter(lambda x: 'message' in x, items))
print(len(items))

16477
5658


In [28]:

origin_question_ids = set()
for item in items:
    if 'origin_question' in item:
        origin_question_ids.add(item['origin_question'])

latest = ''
if message_set:
    latest = str(message_set[-1]['time'])

# cnt_unlabeled = get_unlabeled(course_id)[-1]
cnt_unlabeled = len(list(filter(lambda x: x not in labeled_questions and '[    ]' not in x, set(
    map(lambda x: x['message'], filter(lambda x: x['_id'] in origin_question_ids, message_set))))))
cnt_unlabeled

877